In [1]:
!pip install datasets
!pip install --upgrade accelerate
!pip install transformers[torch]
!pip install evaluate
!pip install bitsandbytes einops wandb -Uqqq
!pip install peft
!pip install torch
!pip install flash_attn
!pip install py_vncorenlp
!pip install vncorenlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 1.6 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 919.7 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 2.3 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.2 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=186973264 sha256=a317a6656be86fa5f583be039c38408258633a21c9de5d5623e11486d6a5541c
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash_attn
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.5 MB/s eta 0:00:0000:010

In [2]:
from datasets import Dataset, load_metric
import accelerate
from torch.utils.data import DataLoader, TensorDataset, Dataset
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoProcessor, TrainingArguments, Trainer
from peft import PeftModel
import requests
import pandas as pd
import numpy as np
import torch
from collections import Counter
import torch.nn.functional as F
from tqdm import tqdm
import re
from torch import nn
import time
import string
from transformers import AutoModel, RobertaForSequenceClassification, Trainer, SchedulerType
from transformers.training_args import TrainingArguments, OptimizerNames
from transformers.trainer_utils import IntervalStrategy
import py_vncorenlp
import json
import random
from vncorenlp import VnCoreNLP

2024-08-07 14:56:12.776009: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 14:56:12.776111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 14:56:12.902229: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Phở Bert

In [3]:
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer

model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")

tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)

# Just like PhoBERT: INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentence = 'Đây là mô_hình rất hay , phù_hợp với điều_kiện và nhu cầu của học sinh, sinh viên .'  

input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    out = model(input_ids)
    probabilities = out.logits.softmax(dim=-1).tolist()[0]

# Get the index of the highest probability
max_index = torch.argmax(out.logits, dim=-1).item()

# Map the index to the corresponding sentiment label
labels = ["Negative", "Positive", "Neutral"]
predicted_label = labels[max_index]

# Print the predicted sentiment
print(f"Predicted sentiment: {predicted_label} with probabilities {probabilities}")


config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Predicted sentiment: Positive with probabilities [0.0019019241444766521, 0.9888592958450317, 0.009238817729055882]


# Dataset

In [4]:
df = pd.read_csv("/kaggle/input/nlp-data/data.csv")
df

,content,label,start
0,Áo bao đẹp ạ!,POS,5
1,Tuyệt vời,POS,5
2,2day ao khong giong trong,NEG,1
3,"Mùi thơm,bôi lên da mềm da",POS,5
4,"Vải đẹp, dày dặn",POS,5
...,...,...,...
31455,Không đáng tiền,NEG,1
31456,Quần rất đẹp,POS,5
31457,Hàng đẹp đúng giá tiền,POS,5
31458,Chất vải khá ổn,POS,4


In [5]:
df[:20]

,content,label,start
0,Áo bao đẹp ạ!,POS,5
1,Tuyệt vời,POS,5
2,2day ao khong giong trong,NEG,1
3,"Mùi thơm,bôi lên da mềm da",POS,5
4,"Vải đẹp, dày dặn",POS,5
5,"Hàng rất đẹp, rất chi là ưng ý",POS,5
6,"Chất lượng sản phẩm tốt, date dài",POS,5
7,Ăn nói và thái độ phục vụ tốt,POS,4
8,Đóng gói sản phẩm chắc chắn,POS,5
9,tất sờn hết ca chưa dùng mà vay r,NEG,2


In [6]:
import regex as re

emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

def clean_text(text):
    text = text.lower()
    text = re.sub(emoji_pattern, " ", text)
    text = re.sub(r'([a-z]+?)\1+',r'\1', text)
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])\s*(\w)", r"\1 \2 \3", text)
    text = re.sub(r"(\w)\s*([" + string.punctuation + "])", r"\1 \2", text)
    # text = re.sub(r"(\d)([^\d.])", r"\1 \2", text)
    # text = re.sub(r"([^\d.])(\d)", r"\1 \2", text)
    text = re.sub(f"([{string.punctuation}])([{string.punctuation}])+",r"\1", text)
    text = text.strip()
    while text.endswith(tuple(string.punctuation+string.whitespace)):
        text = text[:-1]
    while text.startswith(tuple(string.punctuation+string.whitespace)):
        text = text[1:]
    text = re.sub(r"\s+", " ", text)
    return text

def map_label(label):
    label_map = {
        "POS": 1,
        "NEG": 0,
        "NEU": 2
    }
    return label_map[label]

In [7]:
df = df.dropna()

In [8]:
df.loc[:, "content"] = df["content"].map(lambda text: clean_text(text))
df.loc[:, "label"] = df["label"].map(lambda label: map_label(label))

In [9]:
df=df.drop_duplicates("content")
df = df.dropna()
df

,content,label,start
0,áo bao đẹp ạ,1,5
1,tuyệt vời,1,5
2,2day ao khong giong trong,0,1
3,"mùi thơm , bôi lên da mềm da",1,5
4,"vải đẹp , dày dặn",1,5
...,...,...,...
31452,"chất tốt , shop phục vụ rất tốt",1,5
31454,"hàng y hình , đóng gói ko cẩn thận nên quả ngự...",2,3
31457,hàng đẹp đúng giá tiền,1,5
31458,chất vải khá ổn,1,4


In [10]:
df = df.drop(df[df["content"].map(len) < 2].index)

In [11]:
df

,content,label,start
0,áo bao đẹp ạ,1,5
1,tuyệt vời,1,5
2,2day ao khong giong trong,0,1
3,"mùi thơm , bôi lên da mềm da",1,5
4,"vải đẹp , dày dặn",1,5
...,...,...,...
31452,"chất tốt , shop phục vụ rất tốt",1,5
31454,"hàng y hình , đóng gói ko cẩn thận nên quả ngự...",2,3
31457,hàng đẹp đúng giá tiền,1,5
31458,chất vải khá ổn,1,4


In [12]:
from datasets import Dataset
def tokenize_function(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True)

dataset = Dataset.from_pandas(df)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/25608 [00:00<?, ? examples/s]

In [13]:
split_dataset = tokenized_datasets.train_test_split(test_size=0.2)  # 20% for test set

# Access the train and test sets
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print("Train Dataset: ", len(train_dataset))
print("Test Dataset: ", len(test_dataset))

Train Dataset:  20486
Test Dataset:  5122


In [14]:
from collections import Counter

# Đảm bảo rằng bạn đã import hoặc định nghĩa các biến train_dataset và test_dataset
# và tập dữ liệu của bạn có cột nhãn tên là 'labels'

def print_label_distribution(dataset, dataset_name):
    # Lấy danh sách các nhãn từ tập dữ liệu
    labels = [example['label'] for example in dataset]
    
    # Tính toán phân phối nhãn
    label_counts = Counter(labels)
    
    print(f"Label distribution in {dataset_name}:")
    for label, count in label_counts.items():
        print(f"Label {label}: {count} occurrences")

# In phân phối nhãn cho tập huấn luyện và tập kiểm tra
print_label_distribution(train_dataset, "Train Dataset")
print_label_distribution(test_dataset, "Test Dataset")


Label distribution in Train Dataset:
Label 1: 12182 occurrences
Label 2: 3323 occurrences
Label 0: 4981 occurrences
Label distribution in Test Dataset:
Label 1: 3072 occurrences
Label 0: 1234 occurrences
Label 2: 816 occurrences


In [15]:
df["label"].value_counts()

label
1    15254
0     6215
2     4139
Name: count, dtype: int64

# Training

In [16]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [17]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [18]:
from sklearn.metrics import accuracy_score

# Định nghĩa hàm tính toán metrics
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(labels, pred)
    return {"accuracy": accuracy}

In [19]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

# 6. Huấn luyện mô hình
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.460100,0.434312,0.831316
2,0.408500,0.435238,0.839711
3,0.374700,0.446138,0.840882


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=1923, training_loss=0.39868256900686183, metrics={'train_runtime': 1686.8371, 'train_samples_per_second': 36.434, 'train_steps_per_second': 1.14, 'total_flos': 8085212213363712.0, 'train_loss': 0.39868256900686183, 'epoch': 3.0})

In [20]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.44613802433013916,
 'eval_accuracy': 0.840882467786021,
 'eval_runtime': 44.8393,
 'eval_samples_per_second': 114.23,
 'eval_steps_per_second': 3.591,
 'epoch': 3.0}

In [21]:
# Câu cần dự đoán (đã được word-segmented)
sentence = 'sản phẩm này rất tốt'

# Tokenize câu và chuyển đổi thành tensor
input_ids = torch.tensor([tokenizer.encode(sentence)])

# Kiểm tra thiết bị của mô hình và chuyển tensor input_ids sang cùng thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_ids = input_ids.to(device)  # Chuyển tensor input_ids sang cùng thiết bị với mô hình

# Dự đoán nhãn cho câu mới
with torch.no_grad():
    out = model(input_ids)
    probabilities = out.logits.softmax(dim=-1).tolist()[0] 

# Tìm chỉ số của xác suất lớn nhất
max_index = torch.argmax(torch.tensor(probabilities)).item()

# Map chỉ số tới nhãn
labels = ["Negative", "Positive", "Neutral"]
predicted_label = labels[max_index]

# In kết quả xác suất và nhãn dự đoán
print(f"Probabilities: {probabilities}")
print(f"Predicted label: {predicted_label}")


[0.0018679664935916662, 0.9920833110809326, 0.006048749200999737]


Save Model

In [22]:
# Lưu trạng thái của mô hình
torch.save(model.state_dict(), 'model_weights.pth')

# Lưu cấu hình mô hình (nếu cần)
torch.save(model.config, 'model_config.pth')

# Demo

In [23]:
!pip install gradio


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 43.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 66.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.12.5
    Uninstalling tomlkit-0.12.5:
      Successfully uninstalled tomlkit-0.12.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
distributed 2024.5.1 requires dask==2024.5.1, but you have dask 2024.7.0 which is incom

In [24]:
import gradio as gr
def predict(text):
    # Tokenize văn bản
    inputs = tokenizer(text, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    # Di chuyển tensor đến thiết bị phù hợp
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    
    # Dự đoán
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()[0]
    
    # Chuyển đổi kết quả thành chuỗi
    predicted_label = probabilities.argmax()
    return f"Predicted label: {predicted_label}, Probabilities: {probabilities.tolist()}"

# Tạo giao diện Gradio
iface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=2, placeholder="Enter text here..."),
    outputs="text",
    title="RoBERTa Sequence Classification",
    description="A demo for RoBERTa sequence classification model."
)

# Chạy giao diện
iface.launch(share = True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://cb4c4965079632c653.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
